To detect movements in a video, we use background subtraction. There are many different background subtraction algorithms, and each algorithm has various parameters. You can use the default parameter settings in Birdwatcher, but you can also modify the parameters and optimize these for your own videos.

This notebook can be used to play around with all options and various parameters settings of background subtraction algorithms when using Birdwatcher for movement detection, and see how this influences the results.

In [ ]:
import birdwatcher as bw
from birdwatcher.plotting import imshow_frame # birdwatcher has vizualization tools

import matplotlib.pyplot as plt
%matplotlib inline

### Select video fragment

In [ ]:
vfs = bw.VideoFileStream(r'..\videos\zebrafinch.MTS')

Choose a short representative video fragment where the object of interest is moving quite a lot.

In [ ]:
startat = '00:02:10'   # in HOURS:MM:SS
duration = 20          # seconds

In [ ]:
# look at the chosen video fragment
vfs.iter_frames(startat=startat, nframes=vfs.avgframerate*duration).show(framerate=150)

### Choose parameters

In this example, we will use background subtraction MOG2, and try to find optimal parameter values for this algorithm. Also some manipulations before or after performing background subtraction might improve location detection, and therefore, you can also compare the settings of those.

First, decide which settings you would like, by adding various values in the list after each parameter. 

In [ ]:
settings = {'History': [4, 12],
            'ComplexityReductionThreshold': [0.05],
            'BackgroundRatio': [0.1],
            'NMixtures': [7],
            'VarInit': [15],
            'VarMin': [4],
            'VarMax': [75],
            'VarThreshold': [10],
            'VarThresholdGen': [9],
            'DetectShadows': [False],
            'ShadowThreshold': [0.5],
            'ShadowValue': [127],

            'color': [True, False],   # booleans only
            'resizebyfactor': [1],   # use '1' for no change in size
            'blur': [1, 10],   # use '1' for no blur
            'morphologyex': [True, False]}   # booleans only

In [ ]:
all_combinations = bw.get_all_combinations(**settings)
print(f'There are {len(all_combinations)} different combinations of settings to perform movement detection.')

The higher the number of combinations, the longer the next step (running movement detection) will take. Another option is to start by tweaking some parameters with larger steps of parameter values, and fine-tune the values in next rounds.

### Run movemement detection per combination of settings

Movement detection is done for each combination of settings, and the mean coordinate per frame is saved in a Pandas dataframe.

**WARNING:** This step might take a while, depending on the number of settings combinations!

In [ ]:
%%time
params = bw.apply_all_parameters(vfs, settings, startat=startat, duration=duration)
params

Here, you see a pandas dataframe with in the columns all parameters that are used to run movement detection. The rows show the specific value of each parameter and the resulted mean x,y coordinates per frame (NaN means there were no nonzero pixels found for that frame). 

In [ ]:
params.get_info()

### Save ParameterSelection

Save the results as csv file, to be able to load the dataframe later without running all movement detections per setting combination again!

In [ ]:
# save DataFrame
params.save_parameters(f'output/')
params.path

The results are automatically saved in a folder with the name of the videofilestream. Often, several rounds of parameter selection per videofragment will be done with different parameter settings. For this, the same foldername is used, in which case a number is added automatically as suffix to display the round.

If you want to choose your own name for the folder to save the results, you could add the desired foldername as optional argument. Also, you could add overwrite=False to replace an existing ParameterSelection project.

### Load ParameterSelection

In [ ]:
# load ParameterSelection project
params = bw.load_parameterselection(f'output\params_zebrafinch')

Also, the associated videofilestream can be loaded:

In [ ]:
params.vfs

Or watch the videofragment of which the ParameterSelection object is based on:

In [ ]:
frames_fragment = params.get_videofragment()
frames_fragment.show()

To access the data, run:

In [ ]:
params.df

### Correction resizebyfactor

Setting 'resizebyfactor' changes the width and height of the frames. Below, we correct for this change in pixel resolution, so that it's easier to see and compare the effects of different settings on the movementdetection results below.

In [ ]:
df['pixel'] = df['pixel'] / df['resizebyfactor']
df.head()

### Visualize results

Before visualizing the results, look at which settings you could compare.

In [ ]:
# the following settings have been used for backgroundsubstraction in this dataframe
params.get_parameters('all')

In [ ]:
# the following settings have been tested with multiple values
params.get_parameters('multi_only')

Here, you see for which settings multiple values have been used to run movement detection. So, these are also the settings that are interesting to compare in plots or superimpose on the video fragment.

#### Plots

First, choose for each parameter with multiple values which value is the default. TIP: you can copy the output dictionary above and choose one of the values in each list. Use the value of which you think will provide the best location detection. If you have no idea, don't worry, just choose one.

In [ ]:
default_values = {'morphologyex': True,
                  'color': True,
                  'History': 12,
                  'blur': 10}

You can plot the results of two parameters in one figure. The different values of one parameter is outlined in the rows and the other parameter in the columns of the subplots.

In [ ]:
rows = 'History'
cols = 'color'

g = params.plot_parameters(rows, cols, default_values)

To save the plots of all combinations of parameters, use the function below.

In [ ]:
params.batch_plot_parameters(default_values)

The figures are saved in the same directory as where the associated ParameterSelection dataframe is saved. You can go to the folder where the figures are saved and walk through the figures. That way you get a sense of the influence of various parameter-value combination on location detection.

For certain parameters, you might see large noise differences for the different values. For these parameters, choose the best value (the one with the least noise), and use these values as default. Run the above cells again with the new default values. The figures will be saved in a new folder (figures_2). Look again at the figures. Do this several rounds, untill you get an idea of which parameter-value combinations provide the best (least noisy) location detection.

#### Superimpose on video

In [ ]:
# colors in BGR
colors = [('blue', [255, 0, 0]),
          ('orange', [0, 100, 255]),
          ('red', [0, 0, 255]),
          ('lime', [0, 255, 0])]

You can add other colors or change the order of colors in the list. Note that the number of colors defined, is also the maximum number of circles you can draw in the video.

In [ ]:
# choose which settings to compare
compare = {'History': [4, 8, 12]}

# choose for the other settings with multiple values, which value you would like to use for this comparison
other_values = {'color': True,
                'resizebyfactor': 1,
                'blur': 1,
                'morphologyex': True}

df_selection = select_data(df, **other_values)

all_combinations = list(product_dict(**compare))
print(f'There are {len(all_combinations)} combinations of settings to superimpose on a video.')

In [ ]:
def draw_multiple_circles(frames, df, compare, colors):
    
    for i, settings in enumerate(product_dict(**compare)):
        colorname = colors[i][0]
        colorcode = colors[i][1]
        print(f"{colorname} circle: {settings}")

        # select data
        df = select_data(df_selection, **settings)

        # transform into iterable
        iterdata = df.set_index(['framenumber', 'coords']).loc[:, 'pixel'].unstack().values

        frames = frames.draw_circles(iterdata, radius=60, color=colorcode)
    
    return frames

In [ ]:
frames = vfs.iter_frames().draw_framenumbers()
frames = draw_multiple_circles(frames, df_selection, compare, colors)

In [ ]:
# look at the video using show()
frames.show(framerate=100)

In [ ]:
# or, save video with circles superimposed
videoname = vfs.filepath.stem
vfs_circles = frames.tovideo(f'output/{videoname}_multicircles.MTS', framerate=vfs.avgframerate)

It is also possible to get a gradient of the same color. This might be useful if you want to plot circles of different values of the same setting.

In [ ]:
import cv2 as cv

def get_darker_color(color):
    """Get a darker color by decreasing the color value.
    
    color: list of ints
        Should be provided in BGR color space (Blue, Green, Red).
    
    """
    
    bgr = np.uint8([[color]])
    hsv = cv.cvtColor(bgr ,cv.COLOR_BGR2HSV)
    hsv[0][0][2] = hsv[0][0][2] - 50
    new_bgr = cv.cvtColor(hsv ,cv.COLOR_HSV2BGR)
    
    return new_bgr[0][0].tolist()


def get_color_gradientlist(colorname='blue', color=[255, 0, 0]):
    
    gradientlist = [(colorname, color)]
    
    for i in range(1,5):
        color = get_darker_color(color)
        new_name = f'{i}x darker {colorname}'        
        gradientlist.append((new_name, color))
    
    return gradientlist

In [ ]:
blue_colors = get_color_gradientlist(colorname='blue', color=[255, 0, 0])
blue_colors

Note, that there is a maximum of 5 circles.

In [ ]:
frames = vfs.iter_frames().draw_framenumbers()
frames = draw_multiple_circles(frames, df_selection, compare, blue_colors)

In [ ]:
# look at the video using show()
frames.show(framerate=100)